In [10]:
# Bloc 2.0 – Construction d'une série temporelle de base (sans transformation)

import pandas as pd

# 2.0.1 – Vérification que df_sunspots_clean est disponible
try:
    df_sunspots_clean
except NameError:
    raise RuntimeError(
        "df_sunspots_clean n'est pas défini. "
        "Assurez-vous d'avoir exécuté les blocs 1.4 et 1.5 avant ce bloc."
    )

value_col = "Monthly Mean Total Sunspot Number"

# 2.0.2 – Création d'une série temporelle avec Date comme index
df_ts = df_sunspots_clean.copy()
df_ts = df_ts.sort_values("Date").set_index("Date")

# 2.0.3 – Diagnostics simples sur l'index temporel
is_monotonic = df_ts.index.is_monotonic_increasing
n_duplicates = int(df_ts.index.duplicated().sum())
freq = pd.infer_freq(df_ts.index)

print("Résumé de la série temporelle de base :")
print(f"  - Nombre de points            : {len(df_ts)}")
print(f"  - Index monotone croissant   : {is_monotonic}")
print(f"  - Nombre de dates dupliquées : {n_duplicates}")
print(f"  - Fréquence inférée          : {freq}")

print("\nAperçu des 5 premières lignes de df_ts :")
display(df_ts.head())

# 2.0.4 – Logging pour audit
log_message(
    "INFO",
    (
        f"Série temporelle de base construite : n={len(df_ts)}, "
        f"index_monotone={is_monotonic}, dup_dates={n_duplicates}, freq_inferree={freq}"
    ),
    block="BLOC_2.0",
)
log_metric(
    "sunspots_timeseries_length",
    int(len(df_ts)),
    extra={
        "index_monotonic": bool(is_monotonic),
        "n_duplicate_dates": n_duplicates,
        "freq_inferred": freq,
        "value_col": value_col,
    },
)

Résumé de la série temporelle de base :
  - Nombre de points            : 3265
  - Index monotone croissant   : True
  - Nombre de dates dupliquées : 0
  - Fréquence inférée          : ME

Aperçu des 5 premières lignes de df_ts :


,Monthly Mean Total Sunspot Number
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7


[STEP=8][INFO][BLOC_2.0] Série temporelle de base construite : n=3265, index_monotone=True, dup_dates=0, freq_inferree=ME
[METRIC][sunspots_timeseries_length] = 3265 (step=8)


### Bloc 2.1 – Vérification des valeurs manquantes et anomalies simples

Avant de décider de prétraitements (lissage, normalisation, etc.), nous vérifions que la série
temporelle de base ne contient pas de problèmes évidents :

- valeurs manquantes (`NaN`) ;
- valeurs infinies (`+/-inf`) ;
- valeurs négatives éventuelles ;
- nombre de zéros (segments totalement nuls, etc.).

Objectif :

- confirmer que la série brute est exploitable telle quelle ;
- documenter le nombre de zéros et l'éventuelle présence de valeurs étranges,
  sans encore appliquer de transformation (pas de lissage, pas de filtrage).

In [11]:
# Bloc 2.1 – Vérification des valeurs manquantes et anomalies simples

import numpy as np

# 2.1.1 – Vérification df_ts
try:
    df_ts
except NameError:
    raise RuntimeError(
        "df_ts n'est pas défini. Assurez-vous d'avoir exécuté le Bloc 2.0 avant ce bloc."
    )

value_col = "Monthly Mean Total Sunspot Number"
series = df_ts[value_col]

# 2.1.2 – Comptages de base
n_total = len(series)
n_nan = int(series.isna().sum())
n_inf = int(np.isinf(series).sum())
n_neg = int((series < 0).sum())
n_zero = int((series == 0).sum())

val_min = float(series.min())
val_max = float(series.max())
val_mean = float(series.mean())

print("Vérification des valeurs de la série Sunspots :")
print(f"  - Nombre total de points     : {n_total}")
print(f"  - Nombre de NaN              : {n_nan}")
print(f"  - Nombre de +/-inf           : {n_inf}")
print(f"  - Nombre de valeurs < 0      : {n_neg}")
print(f"  - Nombre de valeurs = 0      : {n_zero}")
print(f"  - min                        : {val_min}")
print(f"  - max                        : {val_max}")
print(f"  - moyenne                    : {val_mean}")

# 2.1.3 – Logging
log_message(
    "INFO",
    (
        f"Check valeurs Sunspots : n={n_total}, NaN={n_nan}, inf={n_inf}, "
        f"neg={n_neg}, zeros={n_zero}, min={val_min}, max={val_max}, mean={val_mean}"
    ),
    block="BLOC_2.1",
)
log_metric(
    "sunspots_value_checks",
    int(n_total),
    extra={
        "n_nan": n_nan,
        "n_inf": n_inf,
        "n_neg": n_neg,
        "n_zero": n_zero,
        "val_min": val_min,
        "val_max": val_max,
        "val_mean": val_mean,
    },
)

Vérification des valeurs de la série Sunspots :
  - Nombre total de points     : 3265
  - Nombre de NaN              : 0
  - Nombre de +/-inf           : 0
  - Nombre de valeurs < 0      : 0
  - Nombre de valeurs = 0      : 67
  - min                        : 0.0
  - max                        : 398.2
  - moyenne                    : 81.77877488514548
[STEP=9][INFO][BLOC_2.1] Check valeurs Sunspots : n=3265, NaN=0, inf=0, neg=0, zeros=67, min=0.0, max=398.2, mean=81.77877488514548
[METRIC][sunspots_value_checks] = 3265 (step=9)


### Bloc 2.2 – Décisions initiales de prétraitement

À partir des Blocs 2.0 et 2.1, la série Sunspots présente les caractéristiques suivantes :

- fréquence temporelle mensuelle (`ME`, MonthEnd) ;
- index temporel strictement croissant et sans doublons ;
- aucune valeur manquante (NaN) ni infinie ;
- aucune valeur négative ;
- 67 valeurs égales à 0.0 (minima d'activité).

Décisions méthodologiques **pour l'instant** :

1. **Pas de suppression ni modification des zéros**  
   - Les zéros sont interprétés comme des minima physiques des cycles solaires,
     et non comme des valeurs manquantes.
   - Ils sont conservés tels quels dans la série.

2. **Pas de lissage ni filtrage global**  
   - Aucun lissage (moyennes mobiles, filtres, etc.) n'est appliqué à ce stade.
   - La série de référence pour l’estimation de la dimension `d` sera
     la série brute nettoyée (`df_ts` / [Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0)).

3. **Pas de normalisation globale imposée**  
   - Certaines méthodes d’estimation de `d` peuvent nécessiter une normalisation interne
     (centrage, réduction) pour des raisons numériques.
   - Cette normalisation sera appliquée **localement dans chaque méthode** si nécessaire,
     mais le fichier de base [Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0) demeure non normalisé.

4. **Série de référence Phase 2**  
   - La série de référence pour la Phase 2 est maintenant définie comme :
     [data_phase2/sunspots_clean/Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0)
     (2 colonnes : `Date`, `Monthly Mean Total Sunspot Number`).
   - C’est cette série qui sera utilisée pour :
     - la construction de fenêtres temporelles,
     - l’estimation de `d`,
     - le calcul de `T_log(n, d)`.

Les prétraitements supplémentaires (normalisation dédiée à un algorithme, éventuels lissages de test, etc.)
seront gérés plus tard comme des **versions dérivées explicites**, sauvegardées dans des fichiers séparés
et clairement documentées.

### Bloc 2.2 – Plan des séries et des fenêtres (Phase 2)

Objectif : explorer plusieurs angles **sans perdre le contrôle méthodologique**, en tirant les
leçons de la Phase 1 (sensibilité forte aux choix de prétraitements et aux fenêtres).

#### 2.2.1 Séries considérées

Nous allons travailler avec deux versions de la série Sunspots :

- **S0 – Série brute nettoyée (référence)**  
  - Source : [data_phase2/sunspots_clean/Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0)  
  - Colonnes : `Date`, `Monthly Mean Total Sunspot Number`  
  - Aucune normalisation ni transformation d'échelle globale.

- **S1 – Série normalisée (z-score global)**  
  - Construction : on prend S0, on applique une normalisation (moyenne 0, écart-type 1)
    sur la colonne `Monthly Mean Total Sunspot Number`.  
  - S1 sera sauvegardée dans un CSV séparé, par exemple :  
    `data_phase2/sunspots_clean/Sunspots_clean_zscore.csv`  
  - But : tester la **robustesse** de l’estimation de `d` et de T_log aux changements d’échelle.

(Éventuelles transformations supplémentaires, ex. `log1p`, pourront être ajoutées plus tard,
mais uniquement si cela est justifié et clairement documenté.)

#### 2.2.2 Fenêtres temporelles

Nous définirons plusieurs **tailles de fenêtres** en mois :

- **W60**  :  5 ans   (60 mois)  
- **W132** : ≈11 ans  (132 mois, proche d’un cycle solaire typique)  
- **W264** : ≈22 ans  (264 mois, deux cycles environ)

Ces choix couvrent :

- des fenêtres plus courtes (sensibilité locale),
- un cycle solaire typique,
- une échelle plus longue (superposition de cycles).

#### 2.2.3 Pas de glissement (stride)

Pour chaque taille de fenêtre, nous testerons plusieurs **pas de glissement** (en mois) :

- **G1**  : 1 mois  → suivi très fin dans le temps  
- **G6**  : 6 mois  → intermédiaire  
- **G12** : 12 mois → suivi plus grossier, proche des pas annuels

Ces combinaisons (série, taille de fenêtre, pas) seront :

- construites explicitement,
- sauvegardées dans des fichiers d’artéfacts (par ex. sous `data_phase2/windows/`),
- accompagnées de métadonnées (série utilisée, W, G) pour éviter les confusions.

#### 2.2.4 Rappel méthodologique

- **S0 (brute)** reste la série de **référence physique**.  
- **S1 (z-score)** sert à détecter des artefacts liés uniquement à l’échelle.  
- Pour chaque configuration `(S, W, G)`, nous garderons des logs et des fichiers, afin de
  comprendre si une anomalie vient d’un choix de prétraitement/fenêtrage (problème Phase 1)
  ou de la structure du signal lui-même.

In [12]:
# Bloc 2.3 – Construction et sauvegarde de la série normalisée (S1, z-score)

import pandas as pd
from pathlib import Path

# 2.3.1 – Vérification de df_sunspots_clean
try:
    df_sunspots_clean
except NameError:
    raise RuntimeError(
        "df_sunspots_clean n'est pas défini. "
        "Assurez-vous d'avoir exécuté les blocs 1.5 et 2.0/2.1 avant ce bloc."
    )

value_col = "Monthly Mean Total Sunspot Number"

# 2.3.2 – Calcul du z-score global
mean_val = df_sunspots_clean[value_col].mean()
std_val = df_sunspots_clean[value_col].std()

if std_val == 0:
    raise RuntimeError(
        "L'écart-type de la série est nul, impossible de construire un z-score global."
    )

df_sunspots_z = df_sunspots_clean.copy()
df_sunspots_z[value_col] = (df_sunspots_z[value_col] - mean_val) / std_val

print("Statistiques pour la normalisation z-score :")
print(f"  - moyenne brute : {mean_val}")
print(f"  - écart-type    : {std_val}")

print("\nAperçu des 5 premières lignes de la série normalisée :")
display(df_sunspots_z.head())

# 2.3.3 – Sauvegarde de la série normalisée dans un CSV séparé
DATA_PHASE2_CLEAN_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_clean"
DATA_PHASE2_CLEAN_DIR.mkdir(parents=True, exist_ok=True)

SUNSPOTS_ZSCORE_CSV_PATH = DATA_PHASE2_CLEAN_DIR / "Sunspots_clean_zscore.csv"
df_sunspots_z.to_csv(SUNSPOTS_ZSCORE_CSV_PATH, index=False)

print("\nSérie normalisée (z-score) sauvegardée à l'emplacement :")
print(f"  - {SUNSPOTS_ZSCORE_CSV_PATH}")

# 2.3.4 – Logging pour audit
log_message(
    "INFO",
    (
        f"Série S1 (z-score) créée à partir de S0 : "
        f"mean={mean_val}, std={std_val}, fichier={SUNSPOTS_ZSCORE_CSV_PATH.name}"
    ),
    block="BLOC_2.3",
)
log_metric(
    "sunspots_zscore_created",
    True,
    extra={
        "mean_raw": float(mean_val),
        "std_raw": float(std_val),
        "path": str(SUNSPOTS_ZSCORE_CSV_PATH),
    },
)

Statistiques pour la normalisation z-score :
  - moyenne brute : 81.77877488514548
  - écart-type    : 67.88927651806058

Aperçu des 5 premières lignes de la série normalisée :


,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,0.219788
1,1749-02-28,0.331735
2,1749-03-31,0.514385
3,1749-04-30,0.162341
4,1749-05-31,0.882632



Série normalisée (z-score) sauvegardée à l'emplacement :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_clean\Sunspots_clean_zscore.csv
[STEP=10][INFO][BLOC_2.3] Série S1 (z-score) créée à partir de S0 : mean=81.77877488514548, std=67.88927651806058, fichier=Sunspots_clean_zscore.csv
[METRIC][sunspots_zscore_created] = True (step=10)


### Bloc 2.4 – Construction et traçabilité des fenêtres temporelles (W, G)

Nous construisons maintenant des fenêtres temporelles sur la série mensuelle
(df_ts, fréquence `ME`), avec l’objectif explicite de pouvoir auditer l’impact
des choix de fenêtres sur l’estimation de `d` et de T_log.

#### Fenêtres (W, en mois)

Pour la Phase 2, nous retenons :

- **W60**  :  5 ans   (60 mois)
- **W132** : ≈11 ans  (132 mois, cycle solaire typique)
- **W264** : ≈22 ans  (264 mois, deux cycles environ)

#### Pas de glissement (G, en mois)

Pour chaque W, nous glissons la fenêtre avec :

- **G1**  : 1 mois  → suivi fin
- **G6**  : 6 mois  → suivi intermédiaire
- **G12** : 12 mois → suivi plus grossier (proche pas annuel)

#### Décision méthodologique

- Les fenêtres sont définies **sur l’index temporel de la série brute** (df_ts),
  mais pourront être réutilisées pour S0 (brute) et S1 (z-score), car elles partagent
  le même calendrier.
- Nous ne stockons pas (pour l’instant) toutes les valeurs de chaque fenêtre
  dans des CSV séparés (cela serait très volumineux), mais nous construisons un
  fichier **de définition de fenêtres** avec, pour chaque fenêtre :

  - un identifiant (`window_id`)
  - `window_size_months` (W)
  - `stride_months` (G)
  - `start_index`, `end_index`
  - `start_date`, `end_date`
  - `n_points`

- Ce fichier sera sauvegardé sous `data_phase2/windows/window_definitions.csv` et servira
  de référence unique pour recréer ou analyser les fenêtres dans les blocs d’estimation de `d`
  et de T_log.

Ce design permet d’explorer plusieurs combinaisons (W, G) tout en gardant une traçabilité
claire des choix méthodologiques qui ont posé problème en Phase 1.

In [13]:
# Bloc 2.4 – Construction et sauvegarde des définitions de fenêtres (W, G)

import pandas as pd
from pathlib import Path

# 2.4.1 – Vérification df_ts
try:
    df_ts
except NameError:
    raise RuntimeError(
        "df_ts n'est pas défini. Assurez-vous d'avoir exécuté le Bloc 2.0 avant ce bloc."
    )

idx = df_ts.index
n = len(idx)

print(f"Longueur de la série (df_ts) : {n} points")

# 2.4.2 – Paramètres de fenêtres (en mois)
window_sizes = [60, 132, 264]   # W en mois
strides = [1, 6, 12]            # G en mois

print(f"Tailles de fenêtre (mois) : {window_sizes}")
print(f"Pas de glissement (mois)  : {strides}")

# 2.4.3 – Construction des fenêtres
records = []
window_id = 0

for W in window_sizes:
    for G in strides:
        # glissement sur l'index (en nombre de points, un point = 1 mois)
        start_indices = range(0, max(0, n - W + 1), G)

        nb_windows_for_combo = 0
        for start_idx in start_indices:
            end_idx = start_idx + W - 1
            if end_idx >= n:
                break  # sécurité, normalement couvert par n - W + 1

            start_date = idx[start_idx]
            end_date = idx[end_idx]

            records.append(
                {
                    "window_id": window_id,
                    "window_size_months": W,
                    "stride_months": G,
                    "start_index": start_idx,
                    "end_index": end_idx,
                    "start_date": start_date,
                    "end_date": end_date,
                    "n_points": W,
                }
            )
            window_id += 1
            nb_windows_for_combo += 1

        print(
            f"  - W={W} mois, G={G} mois -> {nb_windows_for_combo} fenêtres construites."
        )

# 2.4.4 – DataFrame des définitions de fenêtres
df_windows = pd.DataFrame.from_records(records)

print(f"\nNombre total de fenêtres (toutes combinaisons W,G) : {len(df_windows)}")
print("\nAperçu des 5 premières fenêtres :")
display(df_windows.head())

# 2.4.5 – Sauvegarde dans data_phase2/windows/window_definitions.csv
WINDOWS_DIR = PHASE2_ROOT / "data_phase2" / "windows"
WINDOWS_DIR.mkdir(parents=True, exist_ok=True)

WINDOW_DEFS_PATH = WINDOWS_DIR / "window_definitions.csv"
df_windows.to_csv(WINDOW_DEFS_PATH, index=False)

print("\nFichier de définitions de fenêtres sauvegardé à l'emplacement :")
print(f"  - {WINDOW_DEFS_PATH}")

# 2.4.6 – Logging
log_message(
    "INFO",
    (
        f"Définitions de fenêtres construites : total={len(df_windows)}, "
        f"window_sizes={window_sizes}, strides={strides}, fichier={WINDOW_DEFS_PATH.name}"
    ),
    block="BLOC_2.4",
)
log_metric(
    "windows_defined_count",
    int(len(df_windows)),
    extra={
        "window_sizes": window_sizes,
        "strides": strides,
        "path": str(WINDOW_DEFS_PATH),
    },
)

Longueur de la série (df_ts) : 3265 points
Tailles de fenêtre (mois) : [60, 132, 264]
Pas de glissement (mois)  : [1, 6, 12]
  - W=60 mois, G=1 mois -> 3206 fenêtres construites.
  - W=60 mois, G=6 mois -> 535 fenêtres construites.
  - W=60 mois, G=12 mois -> 268 fenêtres construites.
  - W=132 mois, G=1 mois -> 3134 fenêtres construites.
  - W=132 mois, G=6 mois -> 523 fenêtres construites.
  - W=132 mois, G=12 mois -> 262 fenêtres construites.
  - W=264 mois, G=1 mois -> 3002 fenêtres construites.
  - W=264 mois, G=6 mois -> 501 fenêtres construites.
  - W=264 mois, G=12 mois -> 251 fenêtres construites.

Nombre total de fenêtres (toutes combinaisons W,G) : 11682

Aperçu des 5 premières fenêtres :


,window_id,window_size_months,stride_months,start_index,end_index,start_date,end_date,n_points
0,0,60,1,0,59,1749-01-31,1753-12-31,60
1,1,60,1,1,60,1749-02-28,1754-01-31,60
2,2,60,1,2,61,1749-03-31,1754-02-28,60
3,3,60,1,3,62,1749-04-30,1754-03-31,60
4,4,60,1,4,63,1749-05-31,1754-04-30,60



Fichier de définitions de fenêtres sauvegardé à l'emplacement :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\windows\window_definitions.csv
[STEP=11][INFO][BLOC_2.4] Définitions de fenêtres construites : total=11682, window_sizes=[60, 132, 264], strides=[1, 6, 12], fichier=window_definitions.csv
[METRIC][windows_defined_count] = 11682 (step=11)


### Synthèse du Bloc 2 – Prétraitements, séries dérivées et fenêtres

Dans ce Bloc 2, nous avons préparé le signal Sunspots pour l’estimation de `d` et de T_log
en explicitant les choix méthodologiques, afin d’éviter les pièges rencontrés en Phase 1.

1. **Séries temporelles de base**

- **S0 – Série brute nettoyée (référence)**  
  - Fichier : [data_phase2/sunspots_clean/Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0)  
  - Colonnes : `Date`, `Monthly Mean Total Sunspot Number`  
  - Aucune normalisation ni transformation globale.

- **S1 – Série normalisée (z-score)**  
  - Fichier : `data_phase2/sunspots_clean/Sunspots_clean_zscore.csv`  
  - Normalisation globale sur `Monthly Mean Total Sunspot Number` :  
    - moyenne brute ≈ 81.78  
    - écart-type ≈ 67.89  
  - But : tester la **robustesse** de `d` et de T_log à un simple changement d’échelle.

2. **Propriétés de la série**

- 3265 points mensuels, fréquence `ME` (fin de mois).  
- Index temporel strictement croissant, aucun doublon.  
- Aucune valeur manquante, aucune valeur infinie, aucune valeur négative.  
- 67 valeurs égales à 0.0 (minima d’activité), conservées telles quelles.

3. **Fenêtres temporelles (W) et glissements (G)**

- Tailles de fenêtres (en mois) :  
  - W60  :  5 ans  
  - W132 : ≈11 ans (cycle solaire)  
  - W264 : ≈22 ans (deux cycles)

- Pas de glissement (en mois) :  
  - G1  : 1 mois  
  - G6  : 6 mois  
  - G12 : 12 mois

- Nombre de fenêtres construites :  
  - W60  : 3206 (G1), 535 (G6), 268 (G12)  
  - W132 : 3134 (G1), 523 (G6), 262 (G12)  
  - W264 : 3002 (G1), 501 (G6), 251 (G12)  
  - **Total** : 11 682 fenêtres

4. **Traçabilité**

- Toutes les fenêtres sont décrites dans :  
  [data_phase2/windows/window_definitions.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/windows/window_definitions.csv:0:0-0:0)  
  avec : `window_id`, `window_size_months`, `stride_months`, `start_index`,
  `end_index`, `start_date`, `end_date`, `n_points`.

Ces choix assurent une base riche pour explorer la sensibilité de `d` et de T_log
aux décisions de prétraitement et de fenêtrage, tout en gardant une traçabilité
précise de chaque configuration.

## Bloc 3 – Estimation de la dimension effective `d`

Dans ce bloc, nous allons estimer la **dimension effective `d`** de la dynamique Sunspots en
utilisant plusieurs méthodes complémentaires, appliquées aux fenêtres définies en Bloc 2
([window_definitions.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/windows/window_definitions.csv:0:0-0:0)). L’objectif est de **quantifier `d` + son incertitude** et de
comprendre comment les choix méthodologiques (prétraitements, fenêtres, hyperparamètres)
influencent les résultats, pour éviter les dérives observées en Phase 1.

### 3.1 Méthodes candidates pour `d`

Nous prévoyons d’explorer au moins les familles de méthodes suivantes :

- **M1 – Dimension de type Levina–Bickel (k‑NN / dimension intrinsèque)**  
  Estimation de la dimension intrinsèque à partir de distances entre points dans
  l’espace des fenêtres (chaque fenêtre = vecteur de taille W).  
  Points critiques : choix de la plage de `k`, stabilité de `d` en fonction de `k`,
  sensibilité au bruit.

- **M2 – Dimension par Participation Ratio / PCA**  
  Pour chaque fenêtre, calcul des valeurs propres de la matrice de covariance (ou
  corrélation) et estimation d’une dimension effective par Participation Ratio / nombre
  de composantes expliquant une fraction donnée de la variance.  
  Points critiques : normalisation (S0 vs S1), choix du seuil de variance expliquée.

- **M3 – Dimension spectrale (pente du spectre)**  
  Estimation de la dimension/slope effective à partir du spectre de puissance sur
  chaque fenêtre (log–log).  
  Points critiques : choix de la bande fréquentielle utilisée pour la régression,
  sensibilité aux choix de lissage spectral.

D’autres méthodes pourront être ajoutées si nécessaire, mais chaque ajout devra être
motivé et documenté de la même façon (hypothèses, hyperparamètres, limites).

### 3.2 Stratégie sur les fenêtres : toutes vs sous‑échantillon

Nous disposons de **11 682 fenêtres** (W60/W132/W264 × G1/G6/G12). Calculer toutes les
méthodes sur toutes les fenêtres d’un coup serait possible mais risqué pour la clarté
méthodologique. Nous adopterons une stratégie en deux étapes :

- **Étape 1 – Sous‑échantillon de calibration**  
  - Sélection contrôlée d’un **sous‑ensemble de fenêtres** (par ex. quelques dizaines
    de fenêtres par couple (W, G), réparties sur toute la période).  
  - Utilisation de ce sous‑échantillon pour :
    - explorer le comportement de chaque méthode (M1, M2, M3) ;
    - choisir les hyperparamètres (plage de `k`, bande fréquentielle, seuils PCA, etc.) ;
    - repérer les pathologies typiques (instabilité de `d`, valeurs aberrantes, cas
      où la méthode échoue).  
  - Tous ces choix seront loggés et documentés (fichiers et Markdown).

- **Étape 2 – Application à toutes les fenêtres avec hyperparamètres fixés**  
  - Une fois les paramètres fixés sur le sous‑échantillon, application des méthodes
    à **l’ensemble des fenêtres pertinentes** (toutes ou un grand sous‑ensemble en W, G).  
  - Stockage des résultats dans des fichiers dédiés (par ex.  
    `data_phase2/d_estimates/d_levina_bickel.csv`,  
    `.../d_participation_ratio.csv`, `.../d_spectral.csv`), avec :
    - `window_id` (référence à [window_definitions.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/windows/window_definitions.csv:0:0-0:0)) ;
    - valeur estimée de `d` ;
    - indicateurs d’incertitude (erreurs, intervalles, flags de qualité).

### 3.3 Rappel méthodologique (liens avec la Phase 1)

- **Objectif** : ne pas “forcer” une unique valeur de `d` mais obtenir :
  - une **distribution de `d`** (par fenêtre, par méthode) ;
  - des **indicateurs de fiabilité** de chaque estimation.  
- **Comparaisons S0 / S1** : la série brute (S0) est la référence physique ;
  la série z‑score (S1) sert à détecter les artefacts liés à l’échelle.  
- **Fenêtres et glissements** : l’impact de W et G sur `d` fait partie du diagnostic,
  et sera analysé explicitement (pas caché).

Les prochaines cellules de code du Bloc 3 commenceront par préparer l’infrastructure
commune d’estimation (`loader` de fenêtres à partir de [window_definitions.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/windows/window_definitions.csv:0:0-0:0),
sélection de sous‑échantillons de calibration, etc.), puis implémenteront une première
méthode (par exemple M1) sur un sous‑ensemble de fenêtres.